# 문장의 유사도 측정
# (1) 문장을 벡터로 만들어 문장간 거리를 계산하여 유사도를 측정한다.
- sklearn이 제공하는 문장을 벡터로 변환하는 함수 CountVectorizer

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
vectorizer = CountVectorizer(min_df=1)

## 훈련용 문장
- 거리를 구하는 것이므로, 지도할 내용이 없다.

In [6]:
contents = ['상처받은 아이들은 너무 일찍 커버려', '내가 상처받은 거 아는 사람 불편해',
           '잘 사는 사람들은 좋은 사람 되기 쉬워', '아무 일도 아니야 괜찮아']

## 형태소 분석
- 한글은 형태소 분석이 필수

In [4]:
from konlpy.tag import Okt

t = Okt()

In [5]:
contents_tokens = [t.morphs(row) for row in contents]
contents_tokens

[['상처', '받은', '아이', '들', '은', '너무', '일찍', '커버', '려'],
 ['내', '가', '상처', '받은', '거', '아는', '사람', '불편해'],
 ['잘', '사는', '사람', '들', '은', '좋은', '사람', '되기', '쉬워'],
 ['아무', '일도', '아니야', '괜찮아']]

## 형태소 분석된 결과를 다시 하나의 문장으로 합치기
- 영어처럼 띄어쓰기를 기준으로 형태소를 구분 가능하게 하는 것

In [9]:
contents_for_vectorize = []

for cont in contents_tokens:
    sen = ''
    for word in cont:
        sen = sen + ' ' + word
        
    contents_for_vectorize.append(sen)
        
contents_for_vectorize

[' 상처 받은 아이 들 은 너무 일찍 커버 려',
 ' 내 가 상처 받은 거 아는 사람 불편해',
 ' 잘 사는 사람 들 은 좋은 사람 되기 쉬워',
 ' 아무 일도 아니야 괜찮아']

## 벡터화 진행

In [10]:
X = vectorizer.fit_transform(contents_for_vectorize)
X

<4x17 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [11]:
num_sam, num_feat = X.shape
num_sam, num_feat

(4, 17)

- 4개의 문장 & 전체 말뭉치의 단어는 17개

## 말뭉치 확인

In [13]:
vectorizer.get_feature_names_out()

array(['괜찮아', '너무', '되기', '받은', '불편해', '사는', '사람', '상처', '쉬워', '아는',
       '아니야', '아무', '아이', '일도', '일찍', '좋은', '커버'], dtype=object)

In [14]:
X.toarray().transpose()

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 2, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0]], dtype=int64)

- 위에 있는 말뭉치 순서대로, 몇번째 문장에 들어있는지를 알려주는 것
- ex) '괜찮아'는 4번째 문장에 있다.

## 테스트용 문장


##### 토큰화

In [15]:
new_post = ['상처받기 싫어 괜찮아']
new_post_tokens = [t.morphs(row) for row in new_post]

new_post_for_vec = []

for con in new_post_tokens:
    sen = ''
    for word in con:
        sen = sen + ' ' + word
        
    new_post_for_vec.append(sen)

new_post_for_vec

[' 상처 받기 싫어 괜찮아']

##### 벡터로 표현
- 말뭉치에 없는 단어는 벡터로 만들지 못한다.

In [16]:
new_post_vec = vectorizer.transform(new_post_for_vec)
new_post_vec.toarray()

array([[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

## 문장 간 거리 구하기
- 기하학 거리 공식 사용

In [20]:
import scipy as sp

def dist_row(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [21]:
dist = [ dist_row(each, new_post_vec) for each in X]
dist

[2.449489742783178, 2.23606797749979, 3.1622776601683795, 2.0]

In [22]:
print('Best post is ', dist.index(min(dist)), ', dist = ', min(dist))
print('Test post is --> ', new_post)
print('Best dist post is --> ', contents[dist.index(min(dist))])

Best post is  3 , dist =  2.0
Test post is -->  ['상처받기 싫어 괜찮아']
Best dist post is -->  아무 일도 아니야 괜찮아


---
# (2) TF-IDF

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore')

In [25]:
X = vectorizer.fit_transform(contents_for_vectorize)
num_sam, num_feat = X.shape
num_sam, num_feat

(4, 17)

In [26]:
X.toarray().transpose()

array([[0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.34431452, 0.40104275, 0.        , 0.        ],
       [0.        , 0.50867187, 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.        , 0.40104275, 0.6191303 , 0.        ],
       [0.34431452, 0.40104275, 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.        , 0.50867187, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.43671931, 0.        , 0.        , 0.        ]])

- 가중치&역가중치가 반영되면서 아까랑 내용이 다르다.

In [27]:
new_post_vec = vectorizer.transform(new_post_for_vec)
new_post_vec.toarray()

array([[0.78528828, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.6191303 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

### 이번엔 두 벡터 사이의 거리를 구할 때, 벡터의 크기를 1로 변경한 후에(normalize) 거리를 구해준다.
- normalize는 한쪽 특성이 두드러지는 것을 막아준다.

In [29]:
def dist_norm(v1, v2):
    v1_norm = v1 / sp.linalg.norm(v1.toarray())
    v2_norm = v2 / sp.linalg.norm(v2.toarray())
    
    delta = v1_norm - v2_norm
    
    return sp.linalg.norm(delta.toarray())

In [30]:
dist = [dist_norm(each, new_post_vec) for each in X]

print('Best post is ', dist.index(min(dist)), ', dist = ', min(dist))
print('Test post is --> ', new_post)
print('Best dist post is --> ', contents[dist.index(min(dist))])

Best post is  3 , dist =  1.1021396119773588
Test post is -->  ['상처받기 싫어 괜찮아']
Best dist post is -->  아무 일도 아니야 괜찮아


In [31]:
dist

[1.254451632446019, 1.2261339938790283, 1.4142135623730951, 1.1021396119773588]

---
# (3) 네이버 지식인 검색 결과에서 유사한 문장 찾기

In [32]:
import urllib.request
import json
import datetime

## 네이버 API 이용 함수

In [37]:
def gen_search_url(api_node, search_text, start_num, disp_num):
    base = 'https://openapi.naver.com/v1/search'
    node = '/' + api_node + '.json'
    param_query = '?query=' + urllib.parse.quote(search_text)
    param_start = '&start=' + str(start_num)
    param_disp = '&display=' + str(disp_num)
    
    
    return base+node+param_query+param_start+param_disp

In [40]:
def get_res_page(url):
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    
    print('[%s] Url Request Success' %datetime.datetime.now())
    
    return json.loads(response.read().decode('utf-8'))

## 데이터 추출

In [99]:
client_id = 'o_mVUTwQlbfQQLw0nflb'
client_secret = 'woo5lphzu5'

url = gen_search_url('kin', '파이썬', 1, 100)
one_res = get_res_page(url)
one_res

[2022-09-22 13:47:45.863219] Url Request Success


{'lastBuildDate': 'Thu, 22 Sep 2022 13:47:45 +0900',
 'total': 241258,
 'start': 1,
 'display': 100,
 'items': [{'title': '<b>파이썬</b> 케이크 도와주세요 퓨ㅠ',
   'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10402&docId=426155555&qb=7YyM7J207I2s&enc=utf8&section=kin.qna&rank=1&search_sort=0&spq=0',
   'description': '이건 <b>파이썬</b> 인가요? <b>파이썬</b> 아니면 <b>파이썬</b> 코드로 생일 케이크 해주려해서 생일 날짜... <b>파이썬</b>이에요. class HBD: def __init__(self, date): self.date = date def Birthday(self, name): print(f&quot;Happy... '},
  {'title': '<b>파이썬</b> 설치하지 않고 인터넷에 검색해서 하는 법',
   'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10402&docId=427083201&qb=7YyM7J207I2s&enc=utf8&section=kin.qna&rank=2&search_sort=0&spq=0',
   'description': '<b>파이썬</b>을 활용하여 주제탐구를 하고 있는데 선생님께서 알려주시면서 인터넷에 뭐라고 치면 <b>파이썬</b>을... research.google.com/?hl=ko 리플잇 https://replit.com/repls &amp; 그리고 온라인 <b>파이썬</b> 코딩으로 검색하면 많이 있습니다. '},
  {'title': '자바신입개발자로 취직 후 <b>파이썬</b>으로 개발 중인데,,',
   'link': 'https://kin.naver.com/q

In [100]:
one_res['items'][0]

{'title': '<b>파이썬</b> 케이크 도와주세요 퓨ㅠ',
 'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10402&docId=426155555&qb=7YyM7J207I2s&enc=utf8&section=kin.qna&rank=1&search_sort=0&spq=0',
 'description': '이건 <b>파이썬</b> 인가요? <b>파이썬</b> 아니면 <b>파이썬</b> 코드로 생일 케이크 해주려해서 생일 날짜... <b>파이썬</b>이에요. class HBD: def __init__(self, date): self.date = date def Birthday(self, name): print(f&quot;Happy... '}

In [101]:
one_res['items'][0]['description']

'이건 <b>파이썬</b> 인가요? <b>파이썬</b> 아니면 <b>파이썬</b> 코드로 생일 케이크 해주려해서 생일 날짜... <b>파이썬</b>이에요. class HBD: def __init__(self, date): self.date = date def Birthday(self, name): print(f&quot;Happy... '

In [102]:
len(one_res['items'])

100

## 태그 제거 함수

In [103]:
def delete_tag(input_str):
    input_str = input_str.replace('<b>', '')
    input_str = input_str.replace('</b>', '')
    
    return input_str

In [104]:
def get_description(pages):
    contents = []
    
    for sentences in pages['items']:
        contents.append(delete_tag(sentences['description']))
        
    
    return contents

In [105]:
contents = get_description(one_res)

In [106]:
contents

['이건 파이썬 인가요? 파이썬 아니면 파이썬 코드로 생일 케이크 해주려해서 생일 날짜... 파이썬이에요. class HBD: def __init__(self, date): self.date = date def Birthday(self, name): print(f&quot;Happy... ',
 '파이썬을 활용하여 주제탐구를 하고 있는데 선생님께서 알려주시면서 인터넷에 뭐라고 치면 파이썬을... research.google.com/?hl=ko 리플잇 https://replit.com/repls &amp; 그리고 온라인 파이썬 코딩으로 검색하면 많이 있습니다. ',
 '... 회사에서 파이썬으로 개발을 해야하는 상황이라 파이썬을 학습하고 있습니다.. 그냥 참고 3년 파이썬 배우고 3년... 일단 파이썬 개발자로 일하시면서 스스로의 성장과 성과에 대해 꾸준히 생각해 보시고 그 양상에 따라 결정하시는... ',
 '파이썬에서 루트를 계산하는거 말고 출력할 때 루트가 보이게 출력하는 법은 없나요..? 도와주세용  수학 문제를... 거라면 파이썬에서는 sympy 라이브러리를 사용해야 한다고 답변드릴 수 있습니다. 아래 그림을 보시면, math... ',
 '파이썬으로 만든 파일?을 열면 특정 프로그램... &apos;파이썬으로 만든 파일을 열면...&apos; 이라는게 파이썬 프로그램으로 실행한다는 얘기죠? 아래 코드와 같이 파이썬에서 다른 실행프로그램을 실행하는... ',
 '파이썬에서 어려운 점이나 어려운 예시 좀 알려주세영... 이건 파이썬 문제가 아닌 재귀를 지원하는 모든... 파이썬은 무수히 많은 라이브러리 들이 있고, 이를... 예제를 원하시는것 같아, 내가 파이썬을 하면서 마들어본... ',
 '파이썬 독학으로 해도 괜찮을지 싶어서요 괜찮다하는분도 있고 파이썬 독학은 절대 못 한다고 하는분들도 있던데.. 그러면 파이썬 독학 말고 도와줄곳을 알아본후에 다니는게 가장 좋을까요?? 혹시 파이썬 독학 해보셨거나 파이썬 독학... ',
 '파이참, 파이썬 둘 다 설

In [107]:
len(contents)

100

## 형태소 분석 

In [108]:
from konlpy.tag import Okt

t = Okt()

In [109]:
con_tokens = [t.morphs(row) for row in contents] 
con_tokens

[['이건',
  '파이썬',
  '인가요',
  '?',
  '파이썬',
  '아니면',
  '파이썬',
  '코드',
  '로',
  '생일',
  '케이크',
  '해주려',
  '해서',
  '생일',
  '날짜',
  '...',
  '파이썬',
  '이에요',
  '.',
  'class',
  'HBD',
  ':',
  'def',
  '__',
  'init',
  '__(',
  'self',
  ',',
  'date',
  '):',
  'self.date',
  '=',
  'date',
  'def',
  'Birthday',
  '(',
  'self',
  ',',
  'name',
  '):',
  'print',
  '(',
  'f',
  '&',
  'quot',
  ';',
  'Happy',
  '...'],
 ['파이썬',
  '을',
  '활용',
  '하여',
  '주',
  '제',
  '탐구',
  '를',
  '하고',
  '있는데',
  '선생님',
  '께서',
  '알려주시면서',
  '인터넷',
  '에',
  '뭐라고',
  '치면',
  '파이썬',
  '을',
  '...',
  'research.google.com/?hl=ko',
  '리플',
  '잇',
  'https://replit.com/repls',
  '&',
  'amp',
  ';',
  '그리고',
  '온라인',
  '파이썬',
  '코딩',
  '으로',
  '검색',
  '하면',
  '많이',
  '있습니다',
  '.'],
 ['...',
  '회사',
  '에서',
  '파이썬',
  '으로',
  '개발',
  '을',
  '해야하는',
  '상황',
  '이라',
  '파이썬',
  '을',
  '학습',
  '하고',
  '있습니다',
  '..',
  '그냥',
  '참고',
  '3년',
  '파이썬',
  '배우고',
  '3년',
  '...',
  '일단',
  '파이썬',
  '개발자',
  '로',
 

In [110]:
len(con_tokens[0])

48

## 형태소마다 띄어쓰기

In [111]:
con_for_vec = []

for con in con_tokens:
    sen = ''
    
    for w in con:
        sen = sen + '' + w
        
    con_for_vec.append(sen)

con_for_vec

['이건파이썬인가요?파이썬아니면파이썬코드로생일케이크해주려해서생일날짜...파이썬이에요.classHBD:def__init__(self,date):self.date=datedefBirthday(self,name):print(f&quot;Happy...',
 '파이썬을활용하여주제탐구를하고있는데선생님께서알려주시면서인터넷에뭐라고치면파이썬을...research.google.com/?hl=ko리플잇https://replit.com/repls&amp;그리고온라인파이썬코딩으로검색하면많이있습니다.',
 '...회사에서파이썬으로개발을해야하는상황이라파이썬을학습하고있습니다..그냥참고3년파이썬배우고3년...일단파이썬개발자로일하시면서스스로의성장과성과에대해꾸준히생각해보시고그양상에따라결정하시는...',
 '파이썬에서루트를계산하는거말고출력할때루트가보이게출력하는법은없나요..?도와주세용수학문제를...거라면파이썬에서는sympy라이브러리를사용해야한다고답변드릴수있습니다.아래그림을보시면,math...',
 '파이썬으로만든파일?을열면특정프로그램...&apos;파이썬으로만든파일을열면...&apos;이라는게파이썬프로그램으로실행한다는얘기죠?아래코드와같이파이썬에서다른실행프로그램을실행하는...',
 '파이썬에서어려운점이나어려운예시좀알려주세영...이건파이썬문제가아닌재귀를지원하는모든...파이썬은무수히많은라이브러리들이있고,이를...예제를원하시는것같아,내가파이썬을하면서마들어본...',
 '파이썬독학으로해도괜찮을지싶어서요괜찮다하는분도있고파이썬독학은절대못한다고하는분들도있던데..그러면파이썬독학말고도와줄곳을알아본후에다니는게가장좋을까요??혹시파이썬독학해보셨거나파이썬독학...',
 '파이참,파이썬둘다설치했는데파이참용량이너무커서요삭제하려고하는데파이썬하나만있어도똑같이...용량이크면다른텍스트에디터사용하시면됩니다.파이썬하나만있으셔도자체적으로사용가능합니다.',
 '제가주피터노트북그걸로파이썬과제를하는데한번만도와주실수있을까요..?내공드릴게요..내공도좋지만..블로그광고클릭이더굳...ㅋㅋㅋㅋ그렇습니다:Dhttps...',
 '교양으로파이썬을듣게됐는

## 백터화

In [112]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(min_df=1)

In [113]:
X = vec.fit_transform(con_for_vec)
X

<100x515 sparse matrix of type '<class 'numpy.int64'>'
	with 571 stored elements in Compressed Sparse Row format>

In [114]:
num_sam, num_feat = X.shape
num_sam, num_feat

(100, 515)

## 새로운 문장 토큰화 & 벡터화

In [115]:
new_post = ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
new_post_tokens = [t.morphs(row) for row in new_post]

new_post_for_vec = []

for content in new_post_tokens:
    sen = ''
    
    for word in content:
        sen = sen + ' ' + word
        
    new_post_for_vec.append(sen)
    
new_post_for_vec

[' 파이썬 을 배우는데 좋은 방법 이 어떤 것 인지 추천 해주세요']

In [116]:
new_post_vec = vec.transform(new_post_for_vec)
new_post_vec.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [117]:
dist = [dist_row(each, new_post_vec) for each in X]
dist

[4.898979485566356,
 3.7416573867739413,
 2.0,
 2.449489742783178,
 3.1622776601683795,
 2.6457513110645907,
 2.0,
 2.23606797749979,
 2.6457513110645907,
 3.605551275463989,
 2.6457513110645907,
 2.8284271247461903,
 2.23606797749979,
 1.7320508075688772,
 2.6457513110645907,
 2.449489742783178,
 1.7320508075688772,
 2.0,
 2.23606797749979,
 2.23606797749979,
 2.449489742783178,
 5.830951894845301,
 2.0,
 2.449489742783178,
 3.4641016151377544,
 12.569805089976535,
 2.449489742783178,
 2.449489742783178,
 2.6457513110645907,
 2.0,
 5.385164807134504,
 3.4641016151377544,
 2.0,
 2.6457513110645907,
 2.6457513110645907,
 2.449489742783178,
 4.123105625617661,
 1.7320508075688772,
 2.0,
 2.23606797749979,
 4.69041575982343,
 2.0,
 2.23606797749979,
 7.54983443527075,
 3.3166247903554,
 2.449489742783178,
 2.0,
 2.23606797749979,
 2.23606797749979,
 2.449489742783178,
 1.7320508075688772,
 2.449489742783178,
 2.449489742783178,
 2.449489742783178,
 5.5677643628300215,
 2.449489742783178,


In [118]:
print('Best post is ', dist.index(min(dist)), ', dist = ', min(dist))
print('Test post is --> ', new_post)
print('Best dist post is --> ', contents[dist.index(min(dist))])

Best post is  80 , dist =  1.4142135623730951
Test post is -->  ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
Best dist post is -->  파이썬코드 조건문으로 만들 수 있는 재밌는 공식 하나 알려주세요 파이썬 저도 배우고 있는 사람으로서 코드도 쉽고 재밌더라구요ㅋㅋ 제가 처음 프로그래밍 언어 배우려고 알아본 곳에서 너무 쉽고 재미있게 알려주셔서... 


## 다른 예문

In [119]:
new_post = ['파이썬을 독학하려는데 좋은 책이 있나요?']
new_post_tokens = [t.morphs(row) for row in new_post]

new_post_for_vec = []

for content in new_post_tokens:
    sen = ''
    
    for word in content:
        sen = sen + ' ' + word
        
    new_post_for_vec.append(sen)
    
new_post_vec = vec.transform(new_post_for_vec)

dist = [dist_row(each, new_post_vec) for each in X]


print('Best post is ', dist.index(min(dist)), ', dist = ', min(dist))
print('Test post is --> ', new_post)
print('Best dist post is --> ', contents[dist.index(min(dist))])

Best post is  80 , dist =  1.4142135623730951
Test post is -->  ['파이썬을 독학하려는데 좋은 책이 있나요?']
Best dist post is -->  파이썬코드 조건문으로 만들 수 있는 재밌는 공식 하나 알려주세요 파이썬 저도 배우고 있는 사람으로서 코드도 쉽고 재밌더라구요ㅋㅋ 제가 처음 프로그래밍 언어 배우려고 알아본 곳에서 너무 쉽고 재미있게 알려주셔서... 


In [121]:
new_post = ['생일케이크']
new_post_tokens = [t.morphs(row) for row in new_post]

new_post_for_vec = []

for content in new_post_tokens:
    sen = ''
    
    for word in content:
        sen = sen + ' ' + word
        
    new_post_for_vec.append(sen)
    
new_post_vec = vec.transform(new_post_for_vec)

dist = [dist_row(each, new_post_vec) for each in X]


print('Best post is ', dist.index(min(dist)), ', dist = ', min(dist))
print('Test post is --> ', new_post)
print('Best dist post is --> ', contents[dist.index(min(dist))])

Best post is  80 , dist =  1.0
Test post is -->  ['생일케이크']
Best dist post is -->  파이썬코드 조건문으로 만들 수 있는 재밌는 공식 하나 알려주세요 파이썬 저도 배우고 있는 사람으로서 코드도 쉽고 재밌더라구요ㅋㅋ 제가 처음 프로그래밍 언어 배우려고 알아본 곳에서 너무 쉽고 재미있게 알려주셔서... 
